In [223]:
import camelot
import pandas as pd
import numpy as np

In [224]:
# https://stackoverflow.com/questions/45846765/efficient-way-to-unnest-explode-multiple-list-columns-in-a-pandas-dataframe

def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [244]:
tables = camelot.read_pdf('Sparkassenrangliste_2018.pdf',pages='all')

# Creating new df to append to
df = pd.DataFrame()

# Iterate over each table in the 10 pages
for table in tables:
    table = table.df
    table.columns = ['Rang','Institut','Sitz','Verband','Bilanzsumme gemäß Bilanzstatistik','Kundeneinlagen','Spareinlagen', 'Kundenkredite','Anzahl Mitarbeiter','Sparkassenstellen (einschließlich SB)']
    table.drop(index=[0,1,2],inplace=True)
    table = table.apply(lambda text: text.str.split('\n'))
    table.reset_index(inplace=True)
    table.drop(columns='index',inplace=True)
    table = explode(table,list(table.columns))
    df = df.append(table)

df.set_index('Rang',inplace=True)

# Get rid of '.' to convert to numeric
df[list(df.columns)[3:]] = df[list(df.columns)[3:]].apply(lambda number: number.str.replace('.',''))

# Convert colukns into integers
for column in list(df.columns)[3:]:
    df[column] = pd.to_numeric(df[column])

# Remove any potential whitespace to have clean text
for column in list(df.columns)[:3]:
    df[column] = df[column].str.strip()
    
# Save df as csv for later use in the scarper
df.to_csv('sparkassen_rangliste_cleaned.csv')

In [381]:
# Extracting sparkassen from df and constructing search url
sparkassen = df['Institut'].to_list()
google_search_url = 'https://www.google.de/search?q='

for sparkasse in sparkassen:
    google_url = google_search_url+sparkasse.replace(' ','+')
    print(google_url)

https://www.google.de/search?q=Hamburger+Sparkasse
https://www.google.de/search?q=Sparkasse+KölnBonn
https://www.google.de/search?q=Kreissparkasse+Köln
https://www.google.de/search?q=Frankfurter+Sparkasse
https://www.google.de/search?q=Stadtsparkasse+München
https://www.google.de/search?q=Sparkasse+Hannover
https://www.google.de/search?q=Mittelbrandenburgische+Sparkasse+in+Potsdam
https://www.google.de/search?q=Sparkasse+Pforzheim+Calw
https://www.google.de/search?q=Ostsächsische+Sparkasse+Dresden
https://www.google.de/search?q=Nassauische+Sparkasse
https://www.google.de/search?q=Kreissparkasse+München+Starnberg+Ebersberg
https://www.google.de/search?q=Stadtsparkasse+Düsseldorf
https://www.google.de/search?q=Die+Sparkasse+Bremen+AG
https://www.google.de/search?q=Sparkasse+Aachen
https://www.google.de/search?q=Sparkasse+Nürnberg
https://www.google.de/search?q=Kreissparkasse+Ludwigsburg
https://www.google.de/search?q=Landessparkasse+zu+Oldenburg
https://www.google.de/search?q=Sparkasse+M

In [332]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver

sparkaseen_links = []

# Setting driver to Safari
driver = webdriver.Chrome(
    executable_path='/Users/felixvemmer/OneDrive/Dokumente/Hobbies/Programming/Python/chromedriver')

for sparkasse in sparkassen:
    google_url = google_search_url+sparkasse.replace(' ','+')
    driver.get(google_url)
    driver.find_element_by_css_selector('h3').click()
    #driver.implicitly_wait(3) 
    target_url = driver.current_url
    sparkaseen_links.append(target_url)

In [ ]:
# Correcting one record manually
df.at['122','links'] = 'https://www.sparkasse-hegau-bodensee.de/de/home.html'

In [394]:
# Creating a list of domains for scrapy
domains = list(df['links'].str.split('/',expand=True)[2].str.split('www.',expand=True)[1])

['haspa.de',
 'sparkasse-koelnbonn.de',
 'ksk-koeln.de',
 'frankfurter-sparkasse.de',
 'sskm.de',
 'sparkasse-hannover.de',
 'mbs.de',
 'sparkasse-pforzheim-calw.de',
 'ostsaechsische-sparkasse-dresden.de',
 'naspa.de',
 'kskmse.de',
 'sskduesseldorf.de',
 'sparkasse-bremen.de',
 'sparkasse-aachen.de',
 'sparkasse-nuernberg.de',
 'ksklb.de',
 'lzo.com',
 'sparkasse-muensterland-ost.de',
 'sparkasse-dortmund.de',
 'sparkasse-heilbronn.de',
 'sparkasse-leipzig.de',
 'ksk-es.de',
 'sparkasse-krefeld.de',
 'kskwn.de',
 'kskbb.de',
 'sparkasse-karlsruhe.de',
 'sparkasse-mainfranken.de',
 'sparkasse-essen.de',
 'sparkasse-westmuensterland.de',
 'sparkasse-heidelberg.de',
 'sparkasse-saarbruecken.de',
 'sparkasse-hgp.de',
 'foerde-sparkasse.de',
 'sparkasse-wuppertal.de',
 'sparkasse-paderborn-detmold.de',
 'sparkasse-bochum.de',
 'nospa.de',
 'sparkasse-osnabrueck.de',
 'sparkasse-bielefeld.de',
 'sparkasse-freiburg.de',
 'sparkasse-neuss.de',
 'sparkasse-holstein.de',
 'sparkasse-ulm.de',
 

In [400]:
 'https://www.sparkasse-koelnbonn.de/de/home/privatkunden/girokonten-und-karten/girokonten.html?'
    'https://www.frankfurter-sparkasse.de/de/home/privatkunden/girokonto-uebersicht.html?'
    'https://www.sskborken.de/de/home/privatkunden/girokonto.html?'
    'https://www.stadtsparkasse-grebenstein.de/de/home/privatkunden/girokonto.html?'
    'https://www.stadtsparkasse-rahden.de/de/home/privatkunden/girokonto.html?'

IndentationError: unexpected indent (<ipython-input-400-aede820bc6d4>, line 2)

In [407]:
start_urls = []

for link in sparkaseen_links:
    base_link = link.replace('.html','/privatkunden/')
    first_pattern = base_link + '/girokonto.html?'
    second_pattern = base_link + '/girokonto-uebersicht.html?'
    third_pattern = base_link + '/girokonten-und-karten/girokonten.html?'
    start_urls.append(first_pattern)
    start_urls.append(second_pattern)
    start_urls.append(third_pattern)

In [409]:
for url in start_urls:
    print(url)

https://www.haspa.de/de/home/privatkunden//girokonto.html?
https://www.haspa.de/de/home/privatkunden//girokonto-uebersicht.html?
https://www.haspa.de/de/home/privatkunden//girokonten-und-karten/girokonten.html?
https://www.sparkasse-koelnbonn.de/de/home/privatkunden//girokonto.html?
https://www.sparkasse-koelnbonn.de/de/home/privatkunden//girokonto-uebersicht.html?
https://www.sparkasse-koelnbonn.de/de/home/privatkunden//girokonten-und-karten/girokonten.html?
https://www.ksk-koeln.de/de/home/privatkunden//girokonto.html?
https://www.ksk-koeln.de/de/home/privatkunden//girokonto-uebersicht.html?
https://www.ksk-koeln.de/de/home/privatkunden//girokonten-und-karten/girokonten.html?
https://www.frankfurter-sparkasse.de/de/home/privatkunden//girokonto.html?
https://www.frankfurter-sparkasse.de/de/home/privatkunden//girokonto-uebersicht.html?
https://www.frankfurter-sparkasse.de/de/home/privatkunden//girokonten-und-karten/girokonten.html?
https://www.sskm.de/de/home/privatkunden//girokonto.ht

In [411]:
test = 'https://www.sskborken.de/de/home/privatkunden/girokonto.html?'

In [418]:
test.split('/de/home')[0] + '/de/home/privatkunden/girokonten-und-karten/girokonten/girodirekt.html?n=true'

'https://www.sskborken.de/de/home/privatkunden/girokonten-und-karten/girokonten/girodirekt.html?n=true'

In [ ]:
'/de/home/privatkunden/girokonten-und-karten/girokonten/girodirekt.html?n=true'